In [55]:
import sys, os, importlib, requests
import rasterio, geojson

import pandas as pd
import geopandas as gpd

import ee
ee.Initialize()

sys.path.append("/home/wb411133/Code/GEE_Zonal/src")
import gee_tools
import gee_helpers

sys.path.append("/home/wb411133/Code/gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

In [2]:
base_folder = "/home/wb411133/data/Country/AFG/"

In [33]:
admin_zones = '/home/wb411133/data/Country/AFG/ADMIN/GeoBoundaries/geoBoundaries-AFG-ADM1.shp'
military_bases = '/home/wb411133/data/Country/AFG/MilitaryBases/fake_bases.shp'

inD = gpd.read_file(admin_zones)
inB = gpd.read_file(military_bases)

ee_features = gee_helpers.gpd_to_gee(inD, 'PROV_34_NA')

In [13]:
# run zonal stats against 
gee_cat = gee_tools.Catalog()

In [20]:
results = gee_cat.search_tags("dmsp")
results.datasets

,id,provider,title,start_date,end_date,startyear,endyear,type,tags,asset_url,thumbnail_url
403,NOAA/DMSP-OLS/CALIBRATED_LIGHTS_V4,"Earth Observation Group, Payne Institute for Public Policy, Colorado School of Mines","DMSP OLS: Global Radiance-Calibrated Nighttime Lights Version 4, Defense Meteorological Program Operational Linescan System",1996-03-16,2011-07-31,1996,2011,image_collection,"[calibrated, dmsp, eog, imagery, lights, nighttime, ols, radiance, visible, yearly]",https://developers.google.com/earth-engine/datasets/catalog/NOAA_DMSP-OLS_CALIBRATED_LIGHTS_V4,https://mw1.google.com/ges/dd/images/NIGHTTIME_sample.png
404,NOAA/DMSP-OLS/NIGHTTIME_LIGHTS,"Earth Observation Group, Payne Institute for Public Policy, Colorado School of Mines","DMSP OLS: Nighttime Lights Time Series Version 4, Defense Meteorological Program Operational Linescan System",1992-01-01,2014-01-01,1992,2014,image_collection,"[dmsp, eog, imagery, lights, nighttime, ols, visible, yearly]",https://developers.google.com/earth-engine/datasets/catalog/NOAA_DMSP-OLS_NIGHTTIME_LIGHTS,https://mw1.google.com/ges/dd/images/NIGHTTIME_sample.png


In [47]:
zs = gee_tools.ZonalStats(
    collection_id = 'NOAA/DMSP-OLS/NIGHTTIME_LIGHTS',
    target_features = ee_features,
    statistic_type = 'sum',
    frequency = 'original',    
)

In [48]:
zs.runZonalStats()

In [49]:
zs.task.status()

AttributeError: 'NoneType' object has no attribute 'status'

In [51]:
s3_base = 's3://wbgdecinternal-ntl/'
ntl_file_list = "/home/wb411133/temp/YEM/AWS_NTL_S3.txt"
focal_tile = "TILE3"
temp_folder = "/home/wb411133/temp"

all_files = []
with open(ntl_file_list, 'r') as in_aws:
    for line in in_aws:
        if focal_tile in line and 'avg_rade9' in line:
            all_files.append(os.path.join(s3_base, line.split(" ")[-1][:-1]))
            
all_files[:5]

['s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201204/TILE3/SVDNB_npp_20120401-20120430_75N060E_vcmcfg_v10_c201605121456.avg_rade9h.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201205/TILE3/SVDNB_npp_20120501-20120531_75N060E_vcmcfg_v10_c201605121458.avg_rade9h.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201206/TILE3/SVDNB_npp_20120601-20120630_75N060E_vcmcfg_v10_c201605121459.avg_rade9h.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201207/TILE3/SVDNB_npp_20120701-20120731_75N060E_vcmcfg_v10_c201605121509.avg_rade9.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201208/TILE3/SVDNB_npp_20120801-20120831_75N060E_vcmcfg_v10_c201602121348.avg_rade9.tif']

In [56]:
for cur_tif in all_files:
    res = rMisc.zonalStats(inD, cur_tif, minVal=0.05)
    res = pd.DataFrame(res,columns=['SUM','MIN','MAX','MEAN'])
    inD[cur_tif.split("/")[5]] = res['SUM']
    tPrint(cur_tif)

13:53:20	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201204/TILE3/SVDNB_npp_20120401-20120430_75N060E_vcmcfg_v10_c201605121456.avg_rade9h.tif
13:53:34	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201205/TILE3/SVDNB_npp_20120501-20120531_75N060E_vcmcfg_v10_c201605121458.avg_rade9h.tif
13:53:47	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201206/TILE3/SVDNB_npp_20120601-20120630_75N060E_vcmcfg_v10_c201605121459.avg_rade9h.tif
13:54:02	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201207/TILE3/SVDNB_npp_20120701-20120731_75N060E_vcmcfg_v10_c201605121509.avg_rade9.tif
13:54:16	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201208/TILE3/SVDNB_npp_20120801-20120831_75N060E_vcmcfg_v10_c201602121348.avg_rade9.tif
13:54:29	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201210/TILE3/SVDNB_npp_20121001-20121031_75N060E_vcmcfg_v10_c201602051401.avg_rade9.tif
13:54:42	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201211/TILE3/SVDNB_npp_20121101-20121130_75N060E_vcmcfg_v10_c201601270845.avg_rade9.

14:06:53	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201704/TILE3/SVDNB_npp_20170401-20170430_75N060E_vcmcfg_v10_c201705011300.avg_rade9.tif
14:07:07	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201705/TILE3/SVDNB_npp_20170501-20170531_75N060E_vcmcfg_v10_c201706021500.avg_rade9.tif
14:07:22	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201706/TILE3/SVDNB_npp_20170601-20170630_75N060E_vcmcfg_v10_c201707021700.avg_rade9h.tif
14:07:36	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201707/TILE3/SVDNB_npp_20170701-20170731_75N060E_vcmcfg_v10_c201708061230.avg_rade9h.tif
14:07:49	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201708/TILE3/SVDNB_npp_20170801-20170831_75N060E_vcmcfg_v10_c201709051000.avg_rade9h.tif
14:08:03	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201709/TILE3/SVDNB_npp_20170901-20170930_75N060E_vcmcfg_v10_c201710041620.avg_rade9h.tif
14:08:17	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201710/TILE3/SVDNB_npp_20171001-20171031_75N060E_vcmcfg_v10_c201711021230.avg_rade9

NameError: name 'in_zones' is not defined

In [58]:
pd.DataFrame(inD.drop(['geometry'], axis=1)).to_csv(f"{admin_zones[:-4]}_NTL.csv")